# Random Forest Algorithm

#### Importación de librerías ⬇️

In [3]:
import pickle
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.tree import export_text, plot_tree
from sklearn.metrics import r2_score, mean_squared_error, root_mean_squared_error
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor  # Asegúrate de importar DecisionTreeRegressor
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

C:\Users\danar\AppData\Local\Temp\ipykernel_9616\3307797748.py:8: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [ ]:
dfs = []

for i in range(2012, 2023):
    df, metadata = pyreadstat.read_sav(f'./data/{i}.sav', encoding="latin1")
    dfs.append(df)

In [4]:
# Importar data general
with open('model.pkl', 'rb') as f:
    data = pickle.load(f)

# Importar data de las 10 enfermedades más comunes
with open('model1.pkl', 'rb') as f:
    data2 = pickle.load(f)
    
print(data)
print(data2)

           AÑO   MES  DIASESTANCIA  SEXO  PPERTENENCIA  EDAD  PERIODOEDA  \
0       2022.0   7.0           1.0   2.0           9.0   1.0         2.0   
1       2022.0   5.0           1.0   2.0           9.0   1.0         3.0   
2       2022.0   4.0           1.0   2.0           9.0   1.0         3.0   
3       2022.0   1.0           1.0   1.0           9.0   1.0         2.0   
4       2022.0   8.0           1.0   2.0           4.0   3.0         3.0   
...        ...   ...           ...   ...           ...   ...         ...   
228303  2021.0   9.0           1.0   2.0           9.0  36.0         3.0   
228304  2021.0   9.0           1.0   1.0           4.0  37.0         3.0   
228305  2021.0   1.0           1.0   1.0           4.0  44.0         3.0   
228306  2021.0   4.0           1.0   1.0           4.0   1.0         3.0   
228307  2021.0  10.0           1.0   1.0           9.0  39.0         3.0   

        DEPTORESIDEN  MUNIRESIDEN  TRATARECIB CAUFIN_CODE  
0               13.0       

In [10]:
def regressionRandomForest(X_train, y_train, profundidad, X):
    param_grid = {
        'n_estimators': [100, 200, 300],
        'max_depth': [None, 10, 20, 30]
    }

    # Crear el modelo de Random Forest
    rf_model = RandomForestRegressor(random_state=42)

    # Crear el objeto GridSearchCV
    grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')
    # Entrenar el modelo
    grid_search.fit(X_train, y_train)

    # Obtener los mejores parámetros y el mejor estimador
    best_params = grid_search.best_params_
    best_estimator = grid_search.best_estimator_

    # Entrenar el mejor modelo en todo el conjunto de entrenamiento
    best_estimator.fit(X_train, y_train)

    # Predecir en el conjunto de prueba
    y_pred = best_estimator.predict(X_test)

    # Calcular el error cuadrático medio
    mse = mean_squared_error(y_test, y_pred)
    print("Error cuadrático medio:", mse)

    # Calcular la raiz del error cuadratico medio
    rmse = root_mean_squared_error(y_test, y_pred)
    print("raiz del error cuadratico medio", rmse)

    # Calcular el coeficiente de determinación
    r2 = r2_score(y_test, y_pred)
    print("Coeficiente de determinación:", r2)





In [13]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score


def classificationRandomForest(X_train, y_train, X_test, y_test):
    param_grid = {
        'n_estimators': [100, 200, 300],
        'max_depth': [None, 10, 20, 30]
    }

    # Crear el modelo de Random Forest
    rf_model = RandomForestClassifier(random_state=42)

    # Crear el objeto GridSearchCV
    grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=5, scoring='accuracy')

    # Entrenar el modelo
    grid_search.fit(X_train, y_train)

    # Obtener los mejores parámetros y el mejor estimador
    best_params = grid_search.best_params_
    best_estimator = grid_search.best_estimator_

    # Entrenar el mejor modelo en todo el conjunto de entrenamiento
    best_estimator.fit(X_train, y_train)

    # Predecir en el conjunto de prueba
    y_pred = best_estimator.predict(X_test)

    # Calcular la precisión del modelo
    accuracy = accuracy_score(y_test, y_pred)
    print("Precisión del modelo:", accuracy)

    # Calcular la precisión, el recall y el puntaje F1
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    print("Precisión:", precision)
    print("Recall:", recall)
    print("Puntaje F1:", f1)

    # Calcular la matriz de confusión
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False, xticklabels=best_estimator.classes_, yticklabels=best_estimator.classes_)
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.title("Confusion Matrix")
    plt.show()

    return best_estimator, accuracy, precision, recall, f1